In [1]:
import torch, numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from datasets import load_dataset
from scipy.optimize import linear_sum_assignment

In [2]:
import torch.nn as nn


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [4]:
def extract_ffn_features_single_pass(model, texts, max_tokens=4000):
    """
    Extract FFN pre-activation features (c_fc outputs)
    for all layers from a SINGLE forward pass,
    excluding padding tokens.
    """
    model.eval()
    features = {i: [] for i in range(len(model.transformer.h))}
    hooks = []

    def make_hook(layer_idx):
        def hook(module, inp, out):
            # out: [batch, seq, d_ff]
            features[layer_idx].append(out.detach().cpu())
        return hook

    # Register hooks ONLY on c_fc
    for i, block in enumerate(model.transformer.h):
        hooks.append(
            block.mlp.c_fc.register_forward_hook(make_hook(i))
        )

    enc = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64
    ).to(device)

    attention_mask = enc["attention_mask"]  # [B, T]

    with torch.no_grad():
        model(**enc)

    for h in hooks:
        h.remove()

    X = {}
    for i in features:
        # [B, T, d]
        # acts = torch.cat(features[i], dim=0)

        # # mask out padding tokens
        # mask = attention_mask.bool().unsqueeze(-1)  # [B, T, 1]
        # acts = acts[mask.expand_as(acts)].view(-1, acts.shape[-1])

        
        acts = torch.cat(features[i], dim=0)  # acts is on CPU

        mask = attention_mask.bool().unsqueeze(-1).cpu()  # <-- FIX

        acts = acts[mask.expand_as(acts)].view(-1, acts.shape[-1])

        if acts.shape[0] > max_tokens:
            acts = acts[:max_tokens]

        X[i] = acts.numpy()

    return X


In [5]:
from scipy.optimize import linear_sum_assignment
import numpy as np

def compute_adjacent_permutations(feature_dict, window_layers):
    """
    Compute permutations ONLY between anchor and adjacent layers
    inside a window.
    """
    anchor = window_layers[0]
    perms = {}

    X_anchor = feature_dict[anchor]

    for layer in window_layers[1:]:
        X_other = feature_dict[layer]

        # Pearson correlation (Eq. 1 in paper)
        C = np.corrcoef(X_anchor, X_other, rowvar=False)
        d = X_anchor.shape[1]
        C = C[:d, d:]

        _, col_ind = linear_sum_assignment(-C)
        perms[layer] = col_ind

    return perms


In [6]:
def sliding_windows(n_layers, k):
    """
    Generate sliding windows of k adjacent layers.
    """
    return [list(range(i, i + k)) for i in range(n_layers - k + 1)]


In [7]:
def permute_ffn(model, layer, perm):
    block = model.transformer.h[layer].mlp
    perm = torch.tensor(perm, dtype=torch.long, device=device)

    # GPT-2 weight shapes are transposed in your setup:
    # c_fc:   [768, 3072]
    # c_proj: [3072, 768]
    with torch.no_grad():
        block.c_fc.weight[:] = block.c_fc.weight[:, perm]
        block.c_fc.bias[:]   = block.c_fc.bias[perm]
        block.c_proj.weight[:] = block.c_proj.weight[perm, :]

In [8]:
import torch
import torch.nn as nn

def merge_k_layers(model, layers, perms):
    """
    Paper-faithful FFN merging with TRUE weight tying.
    - No in-place permutation
    - Logical permutation inside averaging
    - Shared nn.Parameter objects
    """

    anchor = layers[0]
    anchor_mlp = model.transformer.h[anchor].mlp

    device = anchor_mlp.c_fc.weight.device
    k = len(layers)

    # Start from anchor weights
    W_in  = anchor_mlp.c_fc.weight.data.clone()
    b_in  = anchor_mlp.c_fc.bias.data.clone()
    W_out = anchor_mlp.c_proj.weight.data.clone()
    b_out = anchor_mlp.c_proj.bias.data.clone()

    # Accumulate aligned weights (NO mutation)
    for layer in layers[1:]:
        mlp = model.transformer.h[layer].mlp
        perm = torch.tensor(perms[layer], device=device)

        # Logical permutation (as in Eq. 3–6)
        W_in  += mlp.c_fc.weight[:, perm]
        b_in  += mlp.c_fc.bias[perm]
        W_out += mlp.c_proj.weight[perm, :]
        b_out += mlp.c_proj.bias

    # Average
    W_in  /= k
    b_in  /= k
    W_out /= k
    b_out /= k

    # Create SHARED parameters (true tying)
    shared_c_fc_weight   = nn.Parameter(W_in)
    shared_c_fc_bias     = nn.Parameter(b_in)
    shared_c_proj_weight = nn.Parameter(W_out)
    shared_c_proj_bias   = nn.Parameter(b_out)

    # Tie all layers in the window
    for layer in layers:
        mlp = model.transformer.h[layer].mlp

        mlp.c_fc.weight = shared_c_fc_weight
        mlp.c_fc.bias   = shared_c_fc_bias

        mlp.c_proj.weight = shared_c_proj_weight
        mlp.c_proj.bias   = shared_c_proj_bias


In [10]:
import math
import torch

def compute_perplexity(model, texts, batch_size=4, max_length=128):
    model.eval()
    total_nll = 0.0
    total_tokens = 0

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]

        enc = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(device)

        input_ids = enc["input_ids"]
        attention_mask = enc["attention_mask"]

        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits[:, :-1, :]
            labels = input_ids[:, 1:]

        # mask padding tokens
        mask = attention_mask[:, 1:].bool()

        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
        token_log_probs = log_probs.gather(
            dim=-1, index=labels.unsqueeze(-1)
        ).squeeze(-1)

        total_nll += -(token_log_probs[mask]).sum().item()
        total_tokens += mask.sum().item()

    return math.exp(total_nll / total_tokens)

# # merged_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
# # merge_k_layers(merged_model, window, perms)

# merged_ppl = compute_perplexity(model, texts)

# print(f"Merged (window={window}) PPL: {merged_ppl:.2f}")


In [11]:
dataset = load_dataset(
    "wikitext", "wikitext-2-raw-v1", split="validation"
)
texts = [t for t in dataset["text"] if len(t.strip()) > 0][:200]
base_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
base_ppl = compute_perplexity(base_model, texts)

print(f"Baseline GPT-2 PPL: {base_ppl:.2f}")

Baseline GPT-2 PPL: 46.98


In [12]:
# print("Before recovery PPL:",
#           compute_perplexity(model, texts))   

In [13]:
def print_num_layers(model):
    num_layers = len(model.transformer.h)
    print(f"Number of transformer layers: {num_layers}")
print_num_layers(base_model)

Number of transformer layers: 12


In [ ]:
# def count_unique_params(model):
#     seen = set()
#     total = 0
#     for p in model.parameters():
#         if id(p) not in seen:
#             seen.add(id(p))
#             total += p.numel()
#     return total
# def count_params_naive(model):
#     return sum(p.numel() for p in model.parameters())
# def print_param_stats(model, name="model"):
#     naive = count_params_naive(model)
#     unique = count_unique_params(model)
#     print(f"{name}")
#     print(f"  Naive params  : {naive:,}")
#     print(f"  Unique params : {unique:,}")
#     print(f"  Saved params  : {naive - unique:,}")
# print_param_stats(base_model, "Original GPT-2")
# print_param_stats(model, "Merged Model")

Original GPT-2
  Naive params  : 124,439,808
  Unique params : 124,439,808
  Saved params  : 0
Merged Model
  Naive params  : 119,717,376
  Unique params : 119,717,376
  Saved params  : 0


True
True


In [14]:
train_dataset = load_dataset(
    "wikitext", "wikitext-2-raw-v1", split="train[:2%]"
)

train_texts = [t for t in train_dataset["text"] if len(t.strip()) > 0]


In [15]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    enc = tokenizer(
        batch,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )
    return enc

train_loader = DataLoader(
    train_texts,
    batch_size=4,
    shuffle=True,
    collate_fn=collate_fn
)


In [16]:
import torch.nn.functional as F
from torch.optim import AdamW

def recovery_finetune(
    model,
    train_loader,
    steps=1000,
    lr=1e-5,
    weight_decay=0.01,
    device="cuda"
):
    model.train()

    optimizer = AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay
    )

    step = 0
    running_loss = 0.0

    for batch in train_loader:
        if step >= steps:
            break

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids)
        logits = outputs.logits[:, :-1, :]
        labels = input_ids[:, 1:]

        mask = attention_mask[:, 1:].bool()

        log_probs = F.log_softmax(logits, dim=-1)
        token_log_probs = log_probs.gather(
            dim=-1, index=labels.unsqueeze(-1)
        ).squeeze(-1)

        loss = -(token_log_probs[mask]).mean()

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        running_loss += loss.item()
        step += 1

        if step % 100 == 0:
            print(f"[Recovery] step {step} | loss {running_loss / 100:.4f}")
            running_loss = 0.0

    model.eval()


In [17]:
base_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# 1) Extract features ONCE (same forward pass)
feature_mats = extract_ffn_features_single_pass(
    base_model, texts
)

In [18]:
best_ppl = float("inf")
best_window = None
best_state_dict = None

k = 2
windows = sliding_windows(len(base_model.transformer.h), k)

model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
print("Before merging PPL:",
          compute_perplexity(model, texts))

for window in windows:
    model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

    perms = compute_adjacent_permutations(feature_mats, window)
    merge_k_layers(model, window, perms)

    ppl = compute_perplexity(model, texts)
    print(f"Window {window} | PPL {ppl:.2f}")

    if ppl < best_ppl:
        best_ppl = ppl
        best_window = window
        best_state_dict = {
            k: v.detach().cpu().clone()
            for k, v in model.state_dict().items()
        }


best_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
best_model.load_state_dict(best_state_dict)

print(f"\nBEST WINDOW: {best_window}")
print(f"BEST PPL AFTER MERGING: {best_ppl:.2f}")


Before merging PPL: 46.97976365832357
Window [0, 1] | PPL 196.45
Window [1, 2] | PPL 101.00
Window [2, 3] | PPL 86.62
Window [3, 4] | PPL 74.25
Window [4, 5] | PPL 80.09
Window [5, 6] | PPL 92.55
Window [6, 7] | PPL 60.75
Window [7, 8] | PPL 58.43
Window [8, 9] | PPL 57.56
Window [9, 10] | PPL 61.13
Window [10, 11] | PPL 66.76

BEST WINDOW: [8, 9]
BEST PPL AFTER MERGING: 57.56


In [19]:
train_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
train_model.load_state_dict(best_state_dict)

<All keys matched successfully>

In [20]:
print("Before training PPL:",
          compute_perplexity(train_model, texts))

recovery_finetune(
    train_model,
    train_loader,
    steps=10,   # start small
    lr=1e-5,
    device=device
)

print("After training PPL:",
        compute_perplexity(train_model, texts))

Before training PPL: 57.563756951116204
After training PPL: 50.98165648508262


In [21]:
# ---- TEST EVALUATION ----
test_dataset = load_dataset(
    "wikitext", "wikitext-2-raw-v1", split="test"
)

test_texts = [t for t in test_dataset["text"] if len(t.strip()) > 0][:500]

val_ppl = compute_perplexity(train_model, texts)
print(f"Validation PPL after recovery: {val_ppl:.2f}")

test_ppl = compute_perplexity(train_model, test_texts)
print(f"TEST PPL after recovery: {test_ppl:.2f}")


Validation PPL after recovery: 50.98
TEST PPL after recovery: 70.07


In [22]:
print("After training PPL:",
        compute_perplexity(train_model, train_texts))

After training PPL: 67.12803486098032


In [23]:
prompt = "India will become global leader in AI because 1. it has"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    sample_out = train_model.generate(**inputs, max_length=40)
print("\nGenerated output:")
print(tokenizer.decode(sample_out[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated output:
India will become global leader in AI because 1. it has the world's first AI-enabled smart car, and 2. it has the world's first AI-enabled smart car.

The


In [24]:
l0 = model.transformer.h[window[0]].mlp
l1 = model.transformer.h[window[1]].mlp

print(l0.c_fc.weight is l1.c_fc.weight)   # MUST be True
print(l0.c_proj.weight is l1.c_proj.weight)  # MUST be True


True
True


In [ ]:
for window in windows:
    model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

    perms = compute_adjacent_permutations(feature_mats, window)
    merge_k_layers(model, window, perms)

    print("Before recovery PPL:",
          compute_perplexity(model, texts))

    recovery_finetune(
        model,
        train_loader,
        steps=1000,   # start small
        lr=1e-5,
        device=device
    )

    print("After recovery PPL:",
          compute_perplexity(model, texts))
